In [1]:
!pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.5 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

imdb = load_dataset("imdb")

KeyboardInterrupt: ignored

In [ ]:
print(imdb["test"][0])
print(len(imdb["test"]) + len(imdb["train"]))


{'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as 

In [3]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
import os
# ----- path variable ----------
path = "/content/drive/MyDrive/NLP_CW/data"
print(os.path.isdir(path))
# ----- path variable ------

KeyboardInterrupt: ignored

In [ ]:
import csv
data_label = 1
with open('reviews.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Data Label', 'Text']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for folder_path in os.listdir(path):
      current_folder_path = path +"/" + folder_path
      print(f"current folder_path is {current_folder_path}")
      for text_file in os.listdir(current_folder_path):
          with open(os.path.join(current_folder_path, text_file)) as f:
            s = f.read().replace("<br />",". ")
            writer.writerow({'Data Label': data_label, 'Text': s})
      data_label = 0


current folder_path is /content/drive/MyDrive/NLP_CW/data/pos
current folder_path is /content/drive/MyDrive/NLP_CW/data/neg


In [ ]:
from sklearn.model_selection import train_test_split
df_reviews = pd.read_csv('reviews.csv')

df_reviews = df_reviews.sample(frac=1, random_state=1).reset_index(drop=True)

# Group data
grouped = df_reviews.groupby('Data Label')


# Dictionary to store train, test, dev sets for each class
class_sets = {label: {'train': None, 'test': None, 'dev': None} for label, _ in grouped}

# Split each class into train, test, dev sets maintaining the ratio
for label, group in grouped:
   # Split 60-40 initially
    train, test_dev = train_test_split(group, test_size=0.4, random_state=42)
    # Split 50-50 from the remaining 40%
    test, dev = train_test_split(test_dev, test_size=0.5, random_state=42)

    class_sets[label]['train'] = train
    class_sets[label]['test'] = test
    class_sets[label]['dev'] = dev

# Concatenate dataframes for each class to get final train, test, dev sets
train_set = pd.concat([class_sets[label]['train'] for label in class_sets])
dev_set = pd.concat([class_sets[label]['dev'] for label in class_sets])
test_set = pd.concat([class_sets[label]['test'] for label in class_sets])

# Shuffle
train_set = train_set.sample(frac=1, random_state=42).reset_index(drop=True)
dev_set = dev_set.sample(frac=1, random_state=42).reset_index(drop=True)
test_set = test_set.sample(frac=1, random_state=42).reset_index(drop=True)




In [ ]:
imdb = {"train":[],"test":[],"dev":[]}
label = "label"
text = "text"
for index, row in train_set.iterrows():
  c = {label: row["Data Label"], text: row["Text"]}
  imdb["train"].append(c)

for index, row in dev_set.iterrows():
  c = {label: row["Data Label"], text: row["Text"]}
  imdb["dev"].append(c)

for index, row in test_set.iterrows():
  c = {label: row["Data Label"], text: row["Text"]}
  imdb["test"].append(c)

print(len(imdb['train']),len(imdb['dev']),len(imdb['test']))

print(imdb['train'][0])

2400 800 800
{'label': 1, 'text': "Another classic study of the effects of wealth on a southern family is masterfully depicted in Written on the Wind.. . Kyle Hadley has it all. Wealth, a plane, you name it. Kyle's best friend, Mitch, has always gotten him out of difficulty. Mitch finished college, Kyle got thrown out. Mitch is not from a wealthy home. Kyle's family, with Hadley Oil, controls most of everything in the town.. . While in N.Y., Kyle meets the girl of his dreams, nicely played by Lauren Bacall. After a whirlwind romance, he marries her and brings her home. There she meets her father-in-law who warns her how difficult Kyle can be. Kyle sleeps with a gun under his pillow. The Bacall character meets Kyle's sister, Mary Lee, a tramp if ever there were, played to the fullest by Dorothy Malone, who was voted best supporting actress.. . Rock Hudson plays Mitch, the faithful friend.. . A year of wedded bliss for Kyle and his bride ends when Kyle is told by the doctor that he can't

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")

In [ ]:
t = "This was a masterpiece. Not completely faithful to the books, but enthralling from beginning to end. Might be my favorite of the three."
print(tokenizer(t,truncation = True))

{'input_ids': [101, 2023, 2001, 1037, 17743, 1012, 2025, 3294, 11633, 2000, 1996, 2808, 1010, 2021, 4372, 2705, 7941, 2989, 2013, 2927, 2000, 2203, 1012, 2453, 2022, 2026, 5440, 1997, 1996, 2093, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True,max_length=512)

In [ ]:
# tokenized_imdb = imdb.map(preprocess_function, batched=True)
for set_type in imdb:
  for i in range(len(imdb[set_type])):
    review_data = imdb[set_type][i]
    new_data = preprocess_function(review_data)
    imdb[set_type][i]['input_ids'] = new_data['input_ids']
    imdb[set_type][i]['attention_mask'] = new_data['attention_mask']







In [ ]:
print(imdb['train'][2].keys())

dict_keys(['label', 'text', 'input_ids', 'attention_mask'])


In [ ]:
print(tokenized_imdb["train"][34].keys())

NameError: ignored

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:

import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np



def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-cased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!pip install transformers[torch] -U
!pip install accelerate -U


In [ ]:
import torch
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=imdb["train"],
    eval_dataset=imdb["dev"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.356218,0.883750
2,No log,0.418598,0.883750


TrainOutput(global_step=300, training_loss=0.13534924825032552, metrics={'train_runtime': 251.3766, 'train_samples_per_second': 19.095, 'train_steps_per_second': 1.193, 'total_flos': 631666650884928.0, 'train_loss': 0.13534924825032552, 'epoch': 2.0})

In [ ]:
text = "This was a masterpiece. Not completely faithful to the books, but enthralling from beginning to end. Might be my favorite of the three."


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1702003119.976c6afc8ccc.6587.0:   0%|          | 0.00/5.21k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

'https://huggingface.co/seetvn/my_awesome_model/tree/main/'

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="seetvn/my_awesome_model")


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:

result = classifier(text, truncation=True, max_length=512)
print(result)

[{'label': 'NEGATIVE', 'score': 0.5142873525619507}]


In [ ]:
from sklearn.metrics import classification_report

In [ ]:
predictions = []
actual_label = []
count = 0

In [ ]:
test_data = imdb['test'][700:800]
# test_size = (len(test_data))
# print(test_size)

for test in test_data:
  prediction = classifier(test['text'],truncation=True, max_length=512)[0]['label']
  predictions.append(label2id[prediction])
  # actual_label.append(id2label[test['label']])
  actual_label.append(test['label'])
  if prediction == id2label[test['label']]:
    count += 1




# 0.92 + 0.96 + 0.91 + 0.85 + 0.91 +0.91 + 0.84
# 0.90


In [ ]:
print(actual_label)
print(predictions)

[1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1]
[1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1]


In [ ]:
print(classification_report(actual_label,predictions))
print(f"Accuracy is : {count/800}" )

              precision    recall  f1-score   support

           0       0.90      0.88      0.89       400
           1       0.88      0.90      0.89       400

    accuracy                           0.89       800
   macro avg       0.89      0.89      0.89       800
weighted avg       0.89      0.89      0.89       800

Accuracy is : 0.89
